# Lasso regression

In [24]:
library(tidyverse)
library(glmnet)

install.packages('glmnetUtils')
library(glmnetUtils)


Attaching package: ‘glmnetUtils’

The following objects are masked from ‘package:glmnet’:

    cv.glmnet, glmnet



### Read in data and ATAC values, add ATAC values to the dataframe

In [9]:
#all_samples_gaussian_controls <- read.csv("../../../data/cases_controls/all_samples_normalized_CASES.csv", nrows=20000)
#head(all_samples_gaussian_controls)

sample,bin,X30,X31,X32,X33,X34,X35,X36,X37,...,X691,X692,X693,X694,X695,X696,X697,X698,X699,X700
PGDX10344P1,chr10_400,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_401,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_402,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_403,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#all_samples_controls <- all_samples_gaussian_controls

In [10]:
#ATAC <- read.table("../../../data/ATACseq_250kb_bins.txt", header = FALSE)

In [11]:
#head(ATAC)

V1,V2
chr1_0,0.05376791
chr10_0,0.29698782
chr10_1,0.40893151
chr10_10,0.30582283
chr10_100,0.59316570
chr10_101,0.47641373


In [7]:
#all_samples_gaussian_controls <- read.csv("../../../data/all_samples_normalized_CONTROLS.csv", nrows=20000)
all_samples_controls <- readRDS(snakemake@input[["input_train"]])
head(all_samples_controls)

sample,bin,X30,X31,X32,X33,X34,X35,X36,X37,...,X691,X692,X693,X694,X695,X696,X697,X698,X699,X700
PGDX16568P,chr10_400,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX16568P,chr10_40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX16568P,chr10_4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX16568P,chr10_401,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX16568P,chr10_402,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX16568P,chr10_403,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#ATAC <- read.table("../../../data/ATACseq_250kb_bins.txt", header = FALSE)
ATAC <- readRDS(snakemake@input[["input_atac"]])

colnames(ATAC) <- c("bin", "ATAC_observed")
head(ATAC)

ATAC$ATAC_observed <- as.character(ATAC$ATAC_observed)
ATAC$ATAC_observed <- as.numeric(ATAC$ATAC_observed)
ATAC$bin <- as.character(ATAC$bin)

all_samples_ATAC_controls <- inner_join(all_samples_controls, ATAC, by="bin")

head(all_samples_ATAC_controls)

bin,ATAC_observed
chr1_0,0.05376791
chr10_0,0.29698782
chr10_1,0.40893151
chr10_10,0.30582283
chr10_100,0.59316570
chr10_101,0.47641373


sample,bin,X30,X31,X32,X33,X34,X35,X36,X37,...,X692,X693,X694,X695,X696,X697,X698,X699,X700,ATAC_observed
PGDX10344P1,chr10_400,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5866651
PGDX10344P1,chr10_40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.1996134
PGDX10344P1,chr10_4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5968816
PGDX10344P1,chr10_401,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.2905764
PGDX10344P1,chr10_402,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.2319729
PGDX10344P1,chr10_403,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.2262170


In [17]:
ATAC_val <- all_samples_ATAC_controls %>% select(ATAC_observed)
controls_predictions <- all_samples_ATAC_controls %>% select(sample, bin)
all_samples_ATAC_controls = all_samples_ATAC_controls %>% select(-c(ATAC_observed, sample, bin))

head(all_samples_ATAC_controls)
head(ATAC_val)

X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,...,X691,X692,X693,X694,X695,X696,X697,X698,X699,X700
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ATAC_observed
0.5866651
0.1996134
0.5968816
0.2905764
0.2319729
0.2262170


## Lasso regression

## Cross validation to find the best lambda and alpha

## Fit the model on 20% of controls

In [27]:
get_cvm <- function(model) {
   index <- match(model$lambda.min, model$lambda)
  model$cvm[index]
}

In [28]:
y <- ATAC_val %>% as.matrix()
X <- all_samples_ATAC_controls %>% as.matrix()

######### finding the best alpha #########
set.seed(0)
lasso_cva <- cva.glmnet(X, y, nfolds = 10)
enet_performance <- data.frame(alpha = lasso_cva$alpha)
models <- lasso_cva$modlist
enet_performance$cvm <- vapply(models, get_cvm, numeric(1))

minix <- which.min(enet_performance$cvm)
best_alpha <- lasso_cva$alpha[minix]
###########################################

######### finding the best lambda #########
set.seed(0)
lasso_cv <- cv.glmnet(X, y, alpha = best_alpha,
                      standardize = TRUE, nfolds = 10)

plot(lasso_cv)

###########################################


ERROR: Error in saveRDS(fit, snakemake@output[["output_model"]]): object 'snakemake' not found


In [ ]:
(lambda_cv <- lasso_cv$lambda.min)

In [ ]:
y <- ATAC_val %>% as.matrix()
X <- all_samples_ATAC_controls %>% as.matrix()

fit       <- glmnet(X, y, alpha = best_alpha, lambda = lambda_cv, standardize = TRUE)
# saving the model
saveRDS(fit, snakemake@output[["output_model"]])

## Predict on 80% normalized controls and all the cases using the model (fitted on train controls) and save the predictions to the file

In [1]:
#filename = "../../../data/all_samples_normalized_CASES.csv"
#test_data = readRDS("../../data/20_80_splits/20_all_samples_normalized_CONTROLS.rds")

test_data_80_controls = readRDS(snakemake@input[["input_test_controls"]])
test_data_cases = readRDS(snakemake@input[["input_test_cases"]])

In [3]:
head(test_data_80_controls)
head(test_data_cases)

,sample,bin,X30,X31,X32,X33,X34,X35,X36,X37,...,X691,X692,X693,X694,X695,X696,X697,X698,X699,X700
2,PGDX16568P,chr10_40,0,0,0,0,0,0.0000000000,0,0,...,0,0,0,0,0,0,0,0,0,0
9,PGDX16568P,chr10_406,0,0,0,0,0,0.0000000000,0,0,...,0,0,0,0,0,0,0,0,0,0
17,PGDX16568P,chr10_413,0,0,0,0,0,0.0000000000,0,0,...,0,0,0,0,0,0,0,0,0,0
26,PGDX16568P,chr10_421,0,0,0,0,0,0.0001756235,0,0,...,0,0,0,0,0,0,0,0,0,0
30,PGDX16568P,chr10_425,0,0,0,0,0,0.0000000000,0,0,...,0,0,0,0,0,0,0,0,0,0
43,PGDX16568P,chr10_437,0,0,0,0,0,0.0000000000,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data_80_controls_samples_bin_rm <- test_data_80_controls %>% select(-c("sample", "bin")) %>% as.matrix()
pred_80_controls <- as.vector(predict(fit, s=lambda_cv, test_data_80_controls_samples_bin_rm))

test_data_cases_samples_bin_rm <- test_data_cases %>% select(-c("sample", "bin")) %>% as.matrix()
pred_cases <- as.vector(predict(fit, s=lambda_cv, test_data_cases_samples_bin_rm))

In [ ]:
pred_tibble_80_controls <- tibble(sample = test_data_80_controls$sample, bin = test_data_80_controls$bin)
pred_tibble_80_controls <- inner_join(pred_tibble_80_controls, ATAC, by="bin")
pred_tibble_80_controls <- pred_tibble_80_controls %>% mutate(ATAC_predicted = pred_80_controls)

pred_tibble_cases <- tibble(sample = test_data_cases$sample, bin = test_data_cases$bin)
pred_tibble_cases <- inner_join(pred_tibble_cases, ATAC, by="bin")
pred_tibble_cases <- pred_tibble_cases %>% mutate(ATAC_predicted = pred_cases)

pred_tibble <- rbind(pred_tibble_80_controls, pred_tibble_cases)

In [ ]:
ggplot(pred_tibble, aes(x = ATAC_observed, y = ATAC_predicted)) + 
    geom_point(size = 0.5) + 
    geom_smooth(method = "lm", formula = y~x)

## Cor, MAE, RMSE -- all predictions

In [ ]:
cor(pred_tibble$ATAC_observed, pred_tibble$ATAC_predicted)

In [ ]:
ae    <- abs(pred_tibble$ATAC_observed - pred_tibble$ATAC_predicted)
(mae   <- mean(ae))

In [ ]:
se    <- (pred_tibble$ATAC_observed - pred_tibble$ATAC_predicted)^2
mse   <- mean(se)
(rmse <- sqrt(mse))

## Cor, MAE, RMSE -- 80% of the controls predictions

In [ ]:
cor(pred_tibble_80_controls$ATAC_observed, pred_tibble_80_controls$ATAC_predicted)

In [ ]:
ae    <- abs(pred_tibble_80_controls$ATAC_observed - pred_tibble_80_controls$ATAC_predicted)
(mae   <- mean(ae))

In [ ]:
se    <- (pred_tibble_80_controls$ATAC_observed - pred_tibble_80_controls$ATAC_predicted)^2
mse   <- mean(se)
(rmse <- sqrt(mse))

## Cor, MAE, RMSE -- cases predictions

In [ ]:
cor(pred_tibble_cases$ATAC_observed, pred_tibble_cases$ATAC_predicted)

In [ ]:
ae    <- abs(pred_tibble_cases$ATAC_observed - pred_tibble_cases$ATAC_predicted)
(mae   <- mean(ae))

In [ ]:
se    <- (pred_tibble_cases$ATAC_observed - pred_tibble_cases$ATAC_predicted)^2
mse   <- mean(se)
(rmse <- sqrt(mse))

## Save predictions

In [ ]:
saveRDS(pred_tibble, file = snakemake@output[["output_pred"]])